# Poisson Example 2D

Authors: Cian Wilson

## Description

As a reminder, in this case we are seeking the approximate solution to

\begin{equation}
- \nabla^2 T = -\tfrac{5}{4} \exp \left( x+\tfrac{y}{2} \right)
\end{equation}
in a unit square, $\Omega=[0,1]\times[0,1]$, imposing the boundary conditions

\begin{align}
  T &= \exp\left(x+\tfrac{y}{2}\right) && \text{on } \partial\Omega \text{ where } x=0 \text{ or } y=0 \\
  \nabla T\cdot \hat{\vec{n}} &= \exp\left(x + \tfrac{y}{2}\right) && \text{on } \partial\Omega \text{ where } x=1  \\
  \nabla T\cdot \hat{\vec{n}} &= \tfrac{1}{2}\exp\left(x + \tfrac{y}{2}\right) && \text{on } \partial\Omega \text{ where } y=1
 \end{align}

The analytical solution to this problem is $T(x,y) = \exp\left(x+\tfrac{y}{2}\right)$.

## Parallel Scaling

In [the previous notebook](./2.3c_poisson_2d_tests.ipynb) we tested that the error in our [implementation](./2.3b_poisson_2d.ipynb) of a Poisson problem in two-dimensions converged as the number of elements or the polynomial degree increased - a key feature of any numerical scheme.  Another important property of a numerical implementation, particularly in greater than one-dimensional domains, is that they can scale in parallel.  So-called **strong scaling** means that, as more computer processors are used, the time the calculation takes, known as the simulation **wall time**, decreases.  (The alternative, **weak scaling** means that the wall time stays the same if the number of elements is increased proportionally to the number of processors.)

Here we perform strong scaling tests on our function `solve_poisson_2d` from [`notebooks/02_background/2.3b_poisson_2d.ipynb`](./2.3b_poisson_2d.ipynb).  

### Preamble

We start by loading all the modules we will require.

In [ ]:
import sys, os
path = os.path.join(os.path.pardir, os.path.pardir, 'python')
sys.path.append(path)
import utils.ipp
from background.poisson_2d_tests import test_plot_convergence

### Implementation

We perform the strong parallel scaling test using a utility function (from [`python/utils/ipp.py`](../../python/utils/ipp.py)) that loops over a list of the number of processors calling our function for a given number of elements, `ne`, and polynomial order `p`.  It runs our function `solve_poisson_2d` a specified `number` of times and evaluates and returns the time taken for each of a number of requested `steps`.

In [ ]:
# the list of the number of processors we will use
nprocs_scale = [1, 2]

# the number of elements to solve the problem on
ne = 128

# the polynomial degree of our temperature field
p = 2

# perform the calculation a set number of times
number = 2

# We are interested in the time to create the mesh,
# declare the functions, assemble the problem and solve it.
# From our implementation in `solve_poisson_2d` it is also
# possible to request the time to declare the Dirichlet and
# Neumann boundary conditions and the forms.
steps = [
          'Mesh', 'Functions',
          'Assemble', 'Solve',
         ]

# declare a dictionary to store the times each step takes
maxtimes = {}


To start with we test the scaling with the default solver options, which is a direct LU decomposition using the MUMPS library implementation.

In [ ]:
maxtimes['mumps'] = utils.ipp.profile_parallel(nprocs_scale, steps, path, 
                                        'background.poisson_2d', 'solve_poisson_2d',
                                        ne, p, number=number)

We also want to check that the solution is still converging in parallel.  We do this by running our convergence test from [`notebooks/02_background/2.3c_poisson_tests.ipynb`](./2.3c_poisson_2d_tests.ipynb) in parallel using another utility function `utils.ipp.run_parallel`.

In [ ]:
# the list of the number of processors to test the convergence on
nprocs_conv = [2,]
# List of polynomial orders to try
ps = [1, 2]
# List of resolutions to try
nelements = [10, 20, 40, 80, 160]

In [ ]:

errors_l2_all = utils.ipp.run_parallel(nprocs_conv, path, 'background.poisson_2d_tests', 'convergence_errors', ps, nelements)

for errors_l2 in errors_l2_all:
    test_plot_convergence(ps, nelements, errors_l2)